### **Preparing Environment**

In [1]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import re
import nltk
import random
import requests
import numpy as np
import pandas as pd

from fuzzywuzzy import fuzz
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import edit_distance
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### **Dummy Dataset**

In [4]:
# id = list(range(1, 101))

In [5]:
# name = [
#     'Sepatu Sneakers', 'Tas Ransel Stylish', 'Kemeja Flanel Modern', 'Celana Jeans Slim Fit', 'Topi Fedora Elegan',
#     'Kacamata Retro Vintage', 'Jam Tangan Mewah', 'Dompet Kulit Asli', 'Perhiasan Emas Murni', 'Parfum Wangi Floral',
#     'Gelang Fashion Trendy', 'Baju Renang Sporty', 'Topi Baseball Classic', 'Kacamata Hitam Stylish',
#     'Jam Dinding Klasik Vintage', 'Tas Selempang Chic', 'Celana Panjang Casual', 'Kaos Polos Basic', 'Baju Koko Modern',
#     'Sepatu Formal Elegan', 'Tas Makeup Portable', 'Jas Pria Slim Fit', 'Kalung Mutiara Mewah', 'Pashmina Syar\'i Terbaru',
#     'Topi Trucker Trendy', 'Dompet Kartu Minimalis', 'Anting Emas Cantik', 'Lipstik Matte Long-lasting',
#     'Sepatu High Heels Glamor', 'Jaket Kulit Stylish', 'Kemeja Batik Klasik', 'Celana Pendek Denim', 'Kaos Sablon Casual',
#     'Tas Laptop Stylish', 'Blazer Wanita Modern', 'Gelang Berlian Mewah', 'Parfum Pria Maskulin', 'Tas Pesta Glamor',
#     'Kalung Choker Trendy', 'Pashmina Rajut Stylish', 'Topi Keren Stylish', 'Dompet Wanita Stylish', 'Anting Stud Elegant',
#     'Lipstik Glossy Vibrant', 'Sepatu Boots Trendy', 'Sweater Rajut Hangat', 'Celana Jogger Casual', 'Kemeja Putih Formal',
#     'Tas Selempang Kulit', 'Perhiasan Perak Elegan', 'Sepatu Sporty', 'Rok Mini Trendy', 'Baju Kaos Grafis', 'Topi Snapback',
#     'Dompet Lipat Stylish', 'Anting Gantung Unik', 'Lipstik Satin Matte', 'Sepatu Loafer Casual', 'Celana Chino Slim Fit',
#     'Jas Blazer Slim Fit', 'Tas Tote Bag Stylish', 'Gelang Manik-Manik', 'Parfum Floral Fruity', 'Sepatu Sandal Santai',
#     'Blouse Wanita Elegan', 'Sweater Hoodie Sporty', 'Celana Legging Nyaman', 'Kemeja Kotak-Kotak', 'Tas Sling Bag Trendy',
#     'Gelang Kulit Asli', 'Lipstik Velvet Matte', 'Sepatu Running Ringan', 'Baju Kebaya Elegan', 'Topi Beanie Hangat',
#     'Dompet Zipper Minimalis', 'Kalung Liontin Cantik', 'Celana Cargo Praktis', 'Kaos Polo Pria', 'Tas Clutch Glamor',
#     'Gelang Tangan Vintage', 'Parfum Woody Aromatic', 'Sepatu Loafer Formal', 'Rok Midi Trendy', 'Kacamata Oversize Stylish',
#     'Tas Backpack Casual', 'Blouse Peplum Feminin', 'Celana Jogger Denim', 'Kemeja Putih Polos', 'Sepatu Sneakers Trendy',
#     'Topi Bucket Stylish', 'Dompet Lipat Sederhana', 'Anting Hiasan Unik', 'Lipstik Metallic Glamor', 'Sepatu Slip-On Nyaman',
#     'Baju Cardigan Hangat', 'Tas Crossbody Chic', 'Gelang Manik-Manik Etnik', 'Parfum Musk Sensual', 'Sweater Rajut Elegan',
#     'Celana Kulot Modis'
# ]

In [6]:
# description = [
#     'Sepatu Sneakers adalah pilihan yang sempurna untuk gaya kasual Anda. Didesain dengan detail modern dan kenyamanan maksimal.',
#     'Tas Ransel Stylish memberikan gaya dan fungsi yang sempurna. Terbuat dari bahan berkualitas dan dilengkapi dengan banyak kantong.',
#     'Kemeja Flanel Modern memberikan tampilan yang trendy dan nyaman. Cocok untuk berbagai kesempatan dan mudah dipadukan.',
#     'Celana Jeans Slim Fit adalah pilihan yang tepat untuk penampilan yang kasual namun tetap stylish. Terbuat dari bahan denim berkualitas.',
#     'Topi Fedora Elegan memberikan sentuhan klasik yang tak lekang oleh waktu. Cocok untuk melengkapi gaya retro atau modern.',
#     'Kacamata Retro Vintage menambahkan gaya yang unik dan klasik pada penampilan Anda. Terbuat dari bahan berkualitas tinggi.',
#     'Jam Tangan Mewah adalah aksesori yang sempurna untuk menunjukkan keanggunan dan kecanggihan. Didesain dengan detail yang memukau.',
#     'Dompet Kulit Asli memberikan gaya dan fungsi yang tak tertandingi. Terbuat dari kulit asli berkualitas tinggi.',
#     'Perhiasan Emas Murni adalah simbol kemewahan dan keindahan. Terbuat dari emas murni dengan desain yang elegan.',
#     'Parfum Wangi Floral memberikan aroma yang segar dan feminin. Diciptakan dengan komposisi bunga-bungaan yang indah.',
#     'Gelang Fashion Trendy memberikan sentuhan gaya yang modern dan berkelas pada pergelangan tangan Anda. Didesain dengan detail yang menarik.',
#     'Baju Renang Sporty memberikan kenyamanan dan gaya yang tak terbantahkan di kolam renang. Terbuat dari bahan berkualitas dan tahan lama.',
#     'Topi Baseball Classic adalah aksesori yang sempurna untuk melengkapi penampilan santai Anda. Terbuat dari bahan berkualitas dan nyaman dipakai.',
#     'Kacamata Hitam Stylish memberikan perlindungan mata yang maksimal sambil tetap tampil modis. Didesain dengan gaya yang trendy.',
#     'Jam Dinding Klasik Vintage memberikan sentuhan retro yang elegan pada ruangan Anda. Didesain dengan detail yang memukau.',
#     'Tas Selempang Chic memberikan kenyamanan dan gaya yang tak tertandingi. Didesain dengan kompartemen yang cukup luas untuk keperluan sehari-hari.',
#     'Celana Panjang Casual adalah pilihan yang nyaman dan stylish untuk berbagai kesempatan. Terbuat dari bahan berkualitas dan mudah dipadukan.',
#     'Kaos Polos Basic adalah item yang wajib dimiliki dalam koleksi pakaian Anda. Cocok untuk gaya santai atau bisa juga dipadukan dengan aksesori.',
#     'Baju Koko Modern memberikan tampilan yang elegan dan klasik. Cocok untuk berbagai kesempatan formal.',
#     'Sepatu Formal Elegan memberikan tampilan yang anggun dan berkelas pada setiap acara formal. Didesain dengan detail yang sempurna.',
#     'Tas Makeup Portable memudahkan Anda membawa dan menyimpan berbagai perlengkapan makeup dengan praktis. Didesain dengan kompartemen yang fungsional.',
#     'Jas Pria Slim Fit memberikan tampilan yang modern dan berkelas pada setiap acara formal. Didesain dengan detail yang elegan.',
#     'Kalung Mutiara Mewah adalah aksesori yang mewah dan mempesona. Terbuat dari mutiara asli dengan desain yang elegan.',
#     'Pashmina Syar\'i Terbaru memberikan gaya yang modis dan elegan untuk penampilan hijab Anda. Terbuat dari bahan yang nyaman dan mudah diatur.',
#     'Topi Trucker Trendy menambahkan sentuhan gaya yang sporty pada penampilan Anda. Cocok untuk gaya kasual atau acara santai.',
#     'Dompet Kartu Minimalis memberikan kemudahan dan gaya yang minimalis. Terbuat dari bahan berkualitas tinggi.',
#     'Anting Emas Cantik memberikan sentuhan feminin dan elegan pada penampilan Anda. Terbuat dari emas dengan desain yang indah.',
#     'Lipstik Matte Long-lasting memberikan hasil akhir matte yang tahan lama. Dapat memberikan sentuhan warna yang intens dan merona.',
#     'Sepatu High Heels Glamor memberikan ketinggian dan gaya yang anggun pada penampilan Anda. Didesain dengan detail yang memikat.',
#     'Jaket Kulit Stylish memberikan gaya yang cool dan trendi. Terbuat dari kulit berkualitas tinggi dengan desain yang modern.',
#     'Kemeja Batik Klasik memberikan tampilan yang khas dan berkelas. Cocok untuk acara formal atau non-formal.',
#     'Celana Pendek Denim memberikan gaya yang santai dan nyaman untuk cuaca hangat. Terbuat dari bahan denim berkualitas tinggi.',
#     'Kaos Sablon Casual memberikan gaya yang santai dan trendy untuk penampilan sehari-hari. Didesain dengan motif yang menarik.',
#     'Tas Laptop Stylish memberikan perlindungan dan gaya yang maksimal untuk laptop Anda. Terbuat dari bahan berkualitas dan dilengkapi dengan kompartemen yang aman.',
#     'Blazer Wanita Modern memberikan tampilan yang profesional dan stylish. Cocok untuk acara formal atau non-formal.',
#     'Gelang Berlian Mewah memberikan kemewahan dan kilau yang mempesona. Terbuat dari berlian asli dengan desain yang indah.',
#     'Parfum Pria Maskulin memberikan aroma yang maskulin dan elegan. Diciptakan dengan komposisi yang khas untuk pria modern.',
#     'Tas Pesta Glamor memberikan gaya yang mewah dan glamor pada setiap acara pesta. Didesain dengan detail yang memukau.',
#     'Kalung Choker Trendy memberikan sentuhan gaya yang edgy dan modern pada leher Anda. Cocok untuk melengkapi penampilan fashion-forward.',
#     'Pashmina Rajut Stylish memberikan gaya yang modis dan cozy pada penampilan hijab Anda. Terbuat dari bahan rajut berkualitas.',
#     'Topi Keren Stylish memberikan sentuhan gaya yang cool dan trendy pada penampilan Anda. Didesain dengan detail yang menarik.',
#     'Dompet Wanita Stylish memberikan gaya dan fungsi yang tak tertandingi. Terbuat dari bahan berkualitas tinggi.',
#     'Anting Stud Elegant memberikan sentuhan elegan dan klasik pada penampilan Anda. Didesain dengan desain yang timeless.',
#     'Lipstik Glossy Vibrant memberikan hasil akhir glossy yang memikat. Dapat memberikan sentuhan warna yang hidup dan berkilau.',
#     'Sepatu Boots Trendy memberikan gaya yang trendi dan proteksi yang maksimal. Didesain dengan detail yang stylish dan tahan lama.',
#     'Sweater Rajut Hangat memberikan kenyamanan dan gaya yang cozy. Terbuat dari bahan rajut yang hangat dan lembut.',
#     'Celana Jogger Casual memberikan gaya yang santai dan nyaman untuk aktivitas sehari-hari. Terbuat dari bahan yang stretch dan mudah dipadukan.',
#     'Kemeja Putih Formal memberikan tampilan yang klasik dan formal. Cocok untuk berbagai kesempatan yang membutuhkan penampilan yang rapi.',
#     'Tas Selempang Kulit memberikan gaya yang stylish dan fungsional. Terbuat dari kulit berkualitas dengan desain yang modern.',
#     'Perhiasan Perak Elegan memberikan sentuhan elegan dan berkelas pada penampilan Anda. Terbuat dari perak dengan desain yang indah.',
#     'Sepatu Sporty memberikan kenyamanan dan gaya yang tak terbantahkan saat berolahraga. Terbuat dari bahan yang ringan dan nyaman dipakai.',
#     'Rok Mini Trendy memberikan tampilan yang trendy dan playful. Cocok untuk penampilan yang ceria dan fashion-forward.',
#     'Baju Kaos Grafis memberikan gaya yang unik dan ekspresif pada penampilan Anda. Didesain dengan motif grafis yang menarik.',
#     'Topi Snapback memberikan sentuhan gaya yang sporty dan casual pada penampilan Anda. Cocok untuk gaya streetwear atau olahraga.',
#     'Dompet Lipat Stylish memberikan kemudahan dan gaya yang minimalis. Terbuat dari bahan berkualitas tinggi.',
#     'Anting Gantung Unik memberikan sentuhan gaya yang unik dan artistik pada penampilan Anda. Didesain dengan desain yang kreatif.',
#     'Lipstik Satin Matte memberikan hasil akhir satin matte yang elegan. Dapat memberikan sentuhan warna yang lembut dan tahan lama.',
#     'Sepatu Loafer Casual memberikan gaya yang santai dan nyaman pada penampilan Anda. Didesain dengan detail yang stylish.',
#     'Celana Chino Slim Fit memberikan tampilan yang modern dan nyaman. Terbuat dari bahan yang stretch dan mudah dipadukan.',
#     'Jas Blazer Slim Fit memberikan tampilan yang elegan dan berkelas pada setiap acara formal. Didesain dengan detail yang sempurna.',
#     'Tas Tote Bag Stylish memberikan gaya yang chic dan fungsional. Terbuat dari bahan berkualitas tinggi dan dilengkapi dengan tali yang kuat.',
#     'Gelang Manik-Manik memberikan sentuhan gaya yang boho dan berwarna pada pergelangan tangan Anda. Didesain dengan detail yang menarik.',
#     'Parfum Floral Fruity memberikan aroma yang segar dan manis. Diciptakan dengan komposisi bunga dan buah yang lezat.',
#     'Sepatu Sandal Santai memberikan kenyamanan dan gaya yang tak terbantahkan untuk kegiatan santai. Terbuat dari bahan yang nyaman dan ringan.',
#     'Blouse Wanita Elegan memberikan tampilan yang feminin dan elegan. Cocok untuk acara formal atau non-formal.',
#     'Sweater Hoodie Sporty memberikan kenyamanan dan gaya yang sporty. Terbuat dari bahan yang hangat dan dilengkapi dengan hoodie.',
#     'Celana Legging Nyaman memberikan kenyamanan dan fleksibilitas saat digunakan. Terbuat dari bahan yang stretch dan nyaman di kulit.',
#     'Kemeja Kotak-Kotak memberikan tampilan yang klasik dan stylish. Cocok untuk penampilan yang rapi namun tetap casual.',
#     'Tas Sling Bag Trendy memberikan gaya yang trendi dan praktis. Terbuat dari bahan berkualitas tinggi dengan desain yang modern.',
#     'Gelang Kulit Asli memberikan sentuhan gaya yang klasik dan alami pada pergelangan tangan Anda. Terbuat dari kulit asli yang tahan lama.',
#     'Lipstik Velvet Matte memberikan hasil akhir matte yang lembut dan velvety. Dapat memberikan sentuhan warna yang intens dan mewah.',
#     'Sepatu Running Ringan memberikan kenyamanan dan performa yang optimal saat berlari. Terbuat dari bahan yang ringan dan responsif.',
#     'Baju Kebaya Elegan memberikan tampilan yang anggun dan tradisional. Cocok untuk acara formal atau tradisional.',
#     'Topi Beanie Hangat memberikan kenyamanan dan gaya yang hangat pada cuaca dingin. Didesain dengan detail yang stylish.',
#     'Dompet Zipper Minimalis memberikan kemudahan dan gaya yang minimalis. Terbuat dari bahan berkualitas tinggi dengan resleting yang kuat.',
#     'Kalung Liontin Cantik memberikan sentuhan feminin dan elegan pada penampilan Anda. Didesain dengan desain yang manis dan berkilau.',
#     'Celana Cargo Praktis memberikan gaya yang praktis dan fungsional untuk kegiatan sehari-hari. Terbuat dari bahan yang tahan lama.',
#     'Kaos Polo Pria memberikan tampilan yang kasual namun tetap rapi dan stylish. Terbuat dari bahan berkualitas dan nyaman dipakai.',
#     'Tas Clutch Glamor memberikan gaya yang mewah dan glamor pada setiap acara formal. Didesain dengan detail yang memukau.',
#     'Gelang Tangan Vintage memberikan sentuhan retro dan klasik pada pergelangan tangan Anda. Didesain dengan detail yang indah.',
#     'Parfum Woody Aromatic memberikan aroma yang hangat dan maskulin. Diciptakan dengan komposisi kayu dan aroma aromatik yang khas.',
#     'Sepatu Loafer Formal memberikan tampilan yang anggun dan berkelas pada setiap acara formal. Didesain dengan detail yang sempurna.',
#     'Rok Midi Trendy memberikan tampilan yang modern dan stylish. Cocok untuk berbagai kesempatan formal atau non-formal.',
#     'Kacamata Oversize Stylish memberikan gaya yang bold dan fashion-forward pada penampilan Anda. Didesain dengan desain yang statement.',
#     'Tas Backpack Casual memberikan gaya yang casual dan fungsional. Terbuat dari bahan yang tahan lama dan dilengkapi dengan banyak kantong.',
#     'Blouse Peplum Feminin memberikan tampilan yang feminin dan berkelas. Cocok untuk acara formal atau non-formal.',
#     'Celana Jogger Denim memberikan gaya yang trendi dan nyaman untuk aktivitas sehari-hari. Terbuat dari bahan denim yang stretch dan ringan.',
#     'Kemeja Putih Polos memberikan tampilan yang klasik dan universal. Cocok untuk berbagai kesempatan dan mudah dipadukan dengan berbagai pakaian.',
#     'Sepatu Sneakers Trendy memberikan gaya yang trendi dan nyaman untuk penampilan sehari-hari. Terbuat dari bahan yang tahan lama dan dilengkapi dengan sol yang nyaman.',
#     'Topi Bucket Stylish memberikan sentuhan gaya yang casual dan stylish pada penampilan Anda. Didesain dengan desain yang versatile.',
#     'Dompet Lipat Sederhana memberikan kemudahan dan gaya yang sederhana. Terbuat dari bahan berkualitas tinggi dengan desain yang minimalis.',
#     'Anting Hiasan Unik memberikan sentuhan gaya yang unik dan kreatif pada penampilan Anda. Didesain dengan desain yang out-of-the-box.',
#     'Lipstik Metallic Glamor memberikan hasil akhir metallic yang memikat. Dapat memberikan sentuhan warna yang berkilau dan glamor.',
#     'Sepatu Slip-On Nyaman memberikan kenyamanan dan kemudahan saat dipakai. Terbuat dari bahan yang lembut dan elastis.',
#     'Baju Cardigan Hangat memberikan kenyamanan dan gaya yang hangat pada cuaca yang sejuk. Terbuat dari bahan yang lembut dan hangat.',
#     'Tas Crossbody Chic memberikan gaya yang chic dan praktis. Terbuat dari bahan berkualitas tinggi dan dilengkapi dengan tali yang dapat disesuaikan.',
#     'Gelang Manik-Manik Etnik memberikan sentuhan gaya yang etnik dan colorful pada pergelangan tangan Anda. Didesain dengan detail yang artistik.',
#     'Parfum Musk Sensual memberikan aroma yang sensual dan memikat. Diciptakan dengan komposisi musk yang khas dan menarik.',
#     'Sweater Rajut Elegan memberikan kenyamanan dan gaya yang elegan. Terbuat dari bahan rajut yang lembut dan dilengkapi dengan detail yang indah.',
#     'Celana Kulot Modis memberikan tampilan yang modis dan nyaman. Cocok untuk gaya kasual atau acara santai dengan sentuhan yang chic dan trendy'
# ]

In [7]:
# category = [
#     'Sepatu', 'Tas', 'Kemeja', 'Celana', 'Topi',
#     'Kacamata', 'Jam Tangan', 'Dompet', 'Perhiasan', 'Parfum',
#     'Gelang', 'Baju Renang', 'Topi', 'Kacamata',
#     'Jam Dinding', 'Tas', 'Celana', 'Kaos', 'Baju Koko',
#     'Sepatu', 'Tas Makeup', 'Jas', 'Kalung', 'Pashmina',
#     'Topi', 'Dompet Kartu', 'Anting', 'Lipstik',
#     'Sepatu', 'Jaket', 'Kemeja', 'Celana', 'Kaos',
#     'Tas Laptop', 'Blazer', 'Gelang', 'Parfum', 'Tas Pesta',
#     'Kalung', 'Pashmina', 'Topi', 'Dompet', 'Anting',
#     'Lipstik', 'Sepatu', 'Sweater', 'Celana', 'Kemeja',
#     'Tas', 'Perhiasan', 'Sepatu', 'Rok', 'Baju Kaos', 'Topi',
#     'Dompet', 'Anting', 'Lipstik', 'Sepatu', 'Celana',
#     'Jas Blazer', 'Tas', 'Gelang', 'Parfum', 'Sepatu Sandal',
#     'Blouse', 'Sweater', 'Celana', 'Kemeja', 'Tas',
#     'Gelang', 'Lipstik', 'Sepatu', 'Baju Kebaya', 'Topi',
#     'Dompet', 'Kalung', 'Celana', 'Kaos Polo', 'Tas Clutch',
#     'Gelang', 'Parfum', 'Sepatu', 'Rok', 'Kacamata',
#     'Tas', 'Blouse', 'Celana', 'Kemeja', 'Sepatu',
#     'Topi', 'Dompet', 'Anting', 'Lipstik', 'Sepatu',
#     'Baju Cardigan', 'Tas', 'Gelang', 'Parfum', 'Sweater',
#     'Celana'
# ]

In [8]:
# brand = [
#     'Zor Shoes','MaxiBag','Trendy Flannel','SlimFit Denims','Classic Fedora',
#     'RetroVision','Luxe Timepieces','Genuine Wallets','Pure Gold Jewelry','Floral Fragrances',
#     'Fashionable Bracelets','Sporty Swimwear','Basic Baseball Caps','Stylish Sunglasses','Vintage Wall Clocks',
#     'Chic Sling Bags','Casual Trousers','Plain Basics','Modern Koko','Elegant Formal Shoes',
#     'Portable Makeup Cases','SlimFit Men\'s Suits','Luxury Pearl Necklaces','Stylish Pashmina Shawls','Trendy Trucker Hats',
#     'Minimalist Card Wallets','Beautiful Earrings','Long-lasting Matte Lipsticks','Glamorous High Heels',
#     'Stylish Leather Jackets','Classic Batik Shirts','Denim Shorts','Casual Graphic Tees','Stylish Laptop Bags',
#     'Modern Women\'s Blazers','Luxurious Diamond Bracelets','Masculine Perfumes','Glamorous Party Bags','Trendy Choker Necklaces',
#     'Stylish Knitted Pashminas','Cool Snapback Caps','Stylish Women\'s Wallets','Elegant Stud Earrings','Glossy Vibrant Lipsticks',
#     'Trendy Boots','Warm Knitted Sweaters','Casual Joggers','Formal White Shirts','Leather Sling Bags',
#     'Elegant Silver Jewelry','Sporty Shoes','Trendy Mini Skirts','Graphic T-shirts','Stylish Snapback Hats',
#     'Minimalist Fold Wallets','Unique Dangle Earrings','Satin Matte Lipsticks','Casual Loafers','Slim Fit Chinos',
#     'Slim Fit Blazer Jackets','Stylish Tote Bags','Beaded Bracelets','Floral Fruity Perfumes','Casual Sandals',
#     'Elegant Women\'s Blouses','Sporty Hoodie Sweaters','Comfortable Leggings','Classic Checkered Shirts','Trendy Sling Bags',
#     'Genuine Leather Bracelets','Velvet Matte Lipsticks','Lightweight Running Shoes','Elegant Kebaya','Warm Beanie Hats',
#     'Stylish Zipper Wallets','Beautiful Pendant Necklaces','Practical Cargo Pants','Men\'s Polo Shirts','Glamorous Clutches',
#     'Vintage Hand Bracelets','Woody Aromatic Perfumes','Formal Loafers','Trendy Midi Skirts','Oversize Sunglasses',
#     'Casual Backpacks','Feminine Peplum Blouses','Denim Joggers','Plain White Shirts','Sneakers\' Kingdom',
#     'Stylish Belts','Retro Vintage Watches','Urban Messenger Bags','Graceful A-line Skirts','Stylish Jogger Pants','Refined Shirts',
#     'Fashionable Sneakers','Active Sportswear','Casual Jumpsuits','Classic Dress Shirts','Stylish Sandals'
# ]

In [9]:
# tags = [
#     'Oblong, Circle', 'Oval, Oblong', 'Heart, Circle', 'Triangle, Agreeable', 'Square, Openess',
#     'Extraversion, Heart', 'Agreeable, Triangle', 'Oblong, Heart', 'Triangle, Heart', 'Circle, Extraversion',
#     'Conscientious, Circle', 'Oval, Heart', 'Conscientious, Extraversion', 'Openess, Agreeable', 'Heart, Circle',
#     'Circle, Oblong', 'Triangle, Oblong', 'Oblong, Heart, Agreeable', 'Heart, Conscientious', 'Conscientious, Heart',
#     'Heart, Circle, Triangle', 'Conscientious, Circle, Oval', 'Heart, Triangle, Conscientious', 'Triangle, Oval', 'Oblong, Heart',
#     'Agreeable, Circle', 'Extraversion, Triangle', 'Conscientious, Oval', 'Oblong, Circle', 'Heart, Openess',
#     'Triangle, Circle', 'Agreeable, Heart', 'Oblong, Openess', 'Triangle, Extraversion', 'Square, Agreeable',
#     'Circle, Conscientious', 'Heart, Triangle, Openess', 'Oval, Agreeable', 'Heart, Extraversion', 'Circle, Oblong',
#     'Triangle, Conscientious', 'Circle, Neurotic', 'Square, Conscientious', 'Agreeable, Oval', 'Heart, Conscientious, Extraversion',
#     'Triangle, Oblong', 'Oblong, Circle, Conscientious', 'Extraversion, Heart, Circle', 'Agreeable, Openess', 'Heart, Circle, Neurotic',
#     'Triangle, Oval', 'Oblong, Extraversion, Conscientious', 'Agreeable, Circle, Triangle', 'Heart, Openess, Conscientious', 'Circle, Openess',
#     'Triangle, Heart', 'Oblong, Agreeable, Conscientious', 'Extraversion, Circle', 'Conscientious, Openess, Neurotic', 'Heart, Triangle, Oval',
#     'Circle, Oval, Extraversion', 'Oblong, Openess, Agreeable', 'Heart, Conscientious, Circle', 'Triangle, Circle, Extraversion', 'Oblong, Extraversion, Openess',
#     'Agreeable, Heart', 'Extraversion, Openess, Conscientious', 'Circle, Heart, Openess', 'Triangle, Agreeable, Circle', 'Oblong, Circle, Extraversion',
#     'Heart, Circle, Openess', 'Triangle, Extraversion, Conscientious', 'Agreeable, Circle, Neurotic', 'Heart, Triangle, Conscientious', 'Oval, Openess',
#     'Circle, Conscientious, Neurotic', 'Triangle, Circle, Openess', 'Agreeable, Oval, Extraversion', 'Heart, Conscientious, Circle, Neurotic', 'Triangle, Openess',
#     'Oblong, Circle, Triangle', 'Extraversion, Heart, Openess', 'Agreeable, Circle, Openess', 'Heart, Triangle, Circle', 'Oval, Conscientious',
#     'Circle, Triangle, Neurotic', 'Oblong, Extraversion, Neurotic', 'Heart, Openess, Triangle', 'Triangle, Circle, Conscientious', 'Oblong, Circle, Openess',
#     'Heart, Extraversion, Neurotic', 'Agreeable, Conscientious, Openess', 'Circle, Oval, Openess', 'Triangle, Extraversion, Openess', 'Oblong, Heart, Circle, Openess',
#     'Heart, Circle, Extraversion, Conscientious', 'Triangle, Circle, Neurotic', 'Oblong, Extraversion, Agreeable', 'Heart, Triangle, Openess, Conscientious', 'Oval, Circle'
# ]


In [10]:
# merchants = [
#     'Fashion House', 'Beauty Boutique', 'Sports World', 'Tech Zone', 'Home Decor',
#     'Food Paradise', 'Book Haven', 'Music Gallery', 'Pet Shop', 'Artisan Crafts',
#     'Outdoor Gear', 'Kids Kingdom', 'Jewelry Emporium', 'Gadget Hub', 'Wellness Center',
#     'Shoe Emporium', 'Vintage Treasures', 'Gourmet Delights', 'Fitness Zone', 'Game Central',
#     'Fashion Outlet', 'Gift Emporium', 'Home Essentials', 'Health Corner', 'Toy Universe',
#     'Artisan Crafts, Outdoor Gear', 'Tech Zone, Gadget Hub', 'Home Decor, Fashion House', 'Sports World, Fitness Zone',
#     'Book Haven, Music Gallery', 'Beauty Boutique, Wellness Center', 'Food Paradise, Gourmet Delights', 'Jewelry Emporium, Vintage Treasures',
#     'Pet Shop, Toy Universe', 'Kids Kingdom, Outdoor Gear', 'Shoe Emporium, Fashion Outlet', 'Home Essentials, Home Decor',
#     'Artisan Crafts, Wellness Center', 'Game Central, Tech Zone', 'Fashion House, Fashion Outlet', 'Music Gallery, Book Haven',
#     'Outdoor Gear, Fitness Zone', 'Beauty Boutique, Wellness Center, Health Corner', 'Food Paradise, Gourmet Delights, Health Corner', 'Jewelry Emporium, Vintage Treasures, Fashion House',
#     'Pet Shop, Toy Universe, Kids Kingdom', 'Home Decor, Home Essentials, Vintage Treasures', 'Fashion Outlet, Shoe Emporium, Fashion House', 'Sports World, Outdoor Gear, Fitness Zone',
#     'Tech Zone, Gadget Hub, Game Central', 'Artisan Crafts, Wellness Center, Health Corner', 'Book Haven, Music Gallery, Artisan Crafts', 'Beauty Boutique, Wellness Center, Health Corner, Gourmet Delights',
#     'Food Paradise, Gourmet Delights, Health Corner, Fitness Zone', 'Jewelry Emporium, Vintage Treasures, Fashion House, Fashion Outlet', 'Pet Shop, Toy Universe, Kids Kingdom, Outdoor Gear', 'Home Decor, Home Essentials, Vintage Treasures, Fashion Outlet',
#     'Fashion House, Fashion Outlet', 'Sports World, Outdoor Gear', 'Tech Zone, Gadget Hub', 'Home Decor, Fashion House',
#     'Book Haven, Music Gallery', 'Beauty Boutique, Wellness Center', 'Food Paradise, Gourmet Delights', 'Jewelry Emporium, Vintage Treasures',
#     'Pet Shop, Toy Universe', 'Kids Kingdom, Outdoor Gear', 'Shoe Emporium, Fashion Outlet', 'Home Essentials, Home Decor',
#     'Artisan Crafts, Wellness Center', 'Game Central, Tech Zone', 'Music Gallery, Book Haven', 'Fashion House, Fashion Outlet',
#     'Outdoor Gear, Fitness Zone', 'Beauty Boutique, Wellness Center', 'Food Paradise, Gourmet Delights', 'Jewelry Emporium, Vintage Treasures',
#     'Pet Shop, Toy Universe', 'Home Decor, Home Essentials', 'Fashion Outlet, Shoe Emporium', 'Sports World, Outdoor Gear',
#     'Tech Zone, Gadget Hub', 'Artisan Crafts, Wellness Center', 'Book Haven, Music Gallery', 'Beauty Boutique, Wellness Center',
#     'Food Paradise, Gourmet Delights', 'Jewelry Emporium, Vintage Treasures', 'Pet Shop, Toy Universe', 'Home Decor, Home Essentials',
#     'Fashion House, Fashion Outlet', 'Sports World, Outdoor Gear', 'Tech Zone, Gadget Hub', 'Artisan Crafts, Wellness Center',
#     'Game Central, Tech Zone', 'Music Gallery, Book Haven', 'Fashion House, Fashion Outlet', 'Outdoor Gear, Fitness Zone',
#     'Beauty Boutique, Wellness Center', 'Food Paradise, Gourmet Delights', 'Jewelry Emporium, Vintage Treasures'
# ]

In [11]:
# clicked = [random.randint(1, 100) for _ in range(100)]

In [12]:
# # Membuat dataframe dari list
# data = pd.DataFrame({'id': id, 'name': name, 'description' : description, 'category' : category, 'brand': brand, 'tags': tags, 'merchants': merchants, 'clicked': clicked})

# # Menyimpan dataframe ke file CSV
# data.to_csv('data.csv', index=False)

In [13]:
# # Memuat data dari file CSV ke dataframe
# df = pd.read_csv('data.csv')

### **Import Real Dataset**

In [14]:
url = 'https://api.arvigo.site/v1/product-recommendation'
headers = {'X-API-Key': '4a150010-bac7-46e7-8b8b-594f47b0015c'}
response = requests.get(url, headers=headers)

In [15]:
if response.status_code == 200:
    data = response.json()
else:
    print(f"Request failed with status code {response.status_code}")

In [16]:
# Membuat DataFrame dari data
df = pd.DataFrame(data['data'])

### **Understanding Dataset**

In [17]:
# Menampilkan dataframe
df.head()

,id,name,description,category,brand,tags,merchants,clicked
0,1,KAITO Natsuki Signature,SPESIFIKASI\r\n\r\nLebar Frame : 140 mm\r\nLeb...,Glasses,Kcmtku,Square,,9
1,2,MYKI Nial Eyewear,SPESIFIKASI\r\n\r\nLebar Frame : 146 mm\r\nLeb...,Glasses,Kcmtku,Square,,0
2,3,Kaito Cleo Signature,SPESIFIKASI\r\n\r\nLebar Frame : 135 mm\r\nLeb...,Glasses,Kcmtku,Aviator,,0
3,4,Kaito Moana Signature,SPESIFIKASI\r\n\r\nLebar Frame : 138 mm\r\nLeb...,Glasses,Kcmtku,Aviator,,0
4,5,Acetate Cat Eye,Nomor Model: MG01\r\nBahan Lensa: PC\r\nBahan ...,Glasses,Prada,Cat Eye,,0


### **Preprocessing Dataset**

In [18]:
# Menghapus kolom 'clicked'
df = df.drop('clicked', axis=1)

In [19]:
# Mengubah semua nilai pada DataFrame menjadi lower case
df = df.apply(lambda x: x.astype(str).str.lower())

In [20]:
# Mengganti nilai dalam kolom 'tags'
df['tags'] = df['tags'].replace({'circle': 'circle bulat lingkaran',
                                 'heart': 'heart hati',
                                 'square': 'square kotak',
                                 'triangle': 'triangle segitiga',
                                 'extraversion': 'extraversion ekstrover enerjik terbuka sosial',
                                 'neurotic': 'neurotic emosional labil cemas gugup',
                                 'agreeable': 'agreeable ramah sopan bersahabat bergaul',
                                 'conscientious': 'conscientious bertanggung jawab disiplin cermat teliti',
                                 'openness': 'openness terbuka fleksibel kreatif inovatif'}, regex=True)

In [21]:
# Pra-pemrosesan teks
stop_words = set(stopwords.words('indonesian'))
stemmer = PorterStemmer()

In [22]:
def preprocess_text(text):
    # Menghapus karakter khusus
    text = re.sub(r'\W', ' ', text)

    # Tokenisasi
    tokens = word_tokenize(text)

    # Menggabungkan kembali token menjadi kalimat
    processed_text = ' '.join(tokens)
    return processed_text

In [23]:
df['name'] = df['name'].apply(preprocess_text)
df['description'] = df['description'].apply(preprocess_text)

### **Clean Dataset**

In [24]:
df.head()

,id,name,description,category,brand,tags,merchants
0,1,kaito natsuki signature,spesifikasi lebar frame 140 mm lebar lensa 53 ...,glasses,kcmtku,square kotak,
1,2,myki nial eyewear,spesifikasi lebar frame 146 mm lebar lensa 54 ...,glasses,kcmtku,square kotak,
2,3,kaito cleo signature,spesifikasi lebar frame 135 mm lebar lensa 53 ...,glasses,kcmtku,aviator,
3,4,kaito moana signature,spesifikasi lebar frame 138 mm lebar lensa 54 ...,glasses,kcmtku,aviator,
4,5,acetate cat eye,nomor model mg01 bahan lensa pc bahan bingkai ...,glasses,prada,cat eye,


### **Define Function**

In [25]:
def preprocess_query(query):
    # Mengubah huruf menjadi huruf kecil
    query = query.lower()

    # Menghapus tanda baca
    query = re.sub(r'[^a-zA-Z0-9\s]', '', query)
    return query

In [26]:
def check_fuzzy_ratio(query_list, korpus):
    result = []
    for word in query_list:
        match = False
        for k in korpus:
            if fuzz.ratio(word, k) >= 80:
                result.append(k)
                match = True
                break
        if not match:
            result.append('')
    return result

In [27]:
def separate_query(query, result_category, korpus):
    query_list = query.split()
    separated_queries = []
    separated_query = ''
    for i, word in enumerate(query_list):
        if result_category[i-1] != '':
            if separated_query != '':
                separated_queries.append(separated_query.strip())
                separated_query = ''
        if result_category[i] != '':
            separated_query += result_category[i]
        else:
            corrected_word = None
            for k in korpus:
                if fuzz.ratio(word, k) >= 80:
                    corrected_word = k
                    break
            if corrected_word:
                separated_query += corrected_word
            else:
                separated_query += word
        separated_query += ' '
    if separated_query != '':
        separated_queries.append(separated_query.strip())
    return separated_queries

In [28]:
def fix_typo_with_fuzzy_ratio(query, korpus):
    corrected_query = query
    words = query.split()
    for i, word in enumerate(words):
        for keyword in korpus:
            ratio = fuzz.ratio(word, keyword)
            if ratio > 80:
                words[i] = keyword
                break
    corrected_query = ' '.join(words)
    return corrected_query

In [29]:
def identify_keywords(text_list, korpus_category):
    keywords = []
    for text in text_list:
        keyword = ''
        words = text.split()
        for word in words:
            if word in korpus_category:
                keyword = word
                break
        keywords.append(keyword)
    return keywords

In [30]:
def neg_merge_lists(list1, list2):
    merged_list = []
    for i in range(len(list1)):
        if list1[i] == '' or list2[i] == '':
            None
        else:
            merged_list.append(list2[i])
    return merged_list

In [31]:
def pos_merge_lists(list1, list2, list3):
    pos_list1 = []
    pos_list2 = []
    for i in range(len(list1)):
        if list1[i] != '' and list3[i] == '':
            pos_list1.append(list1[i])
        if list2[i] != '' and list3[i] == '':
            pos_list2.append(list2[i])
    return pos_list1, pos_list2

### **Define Important Words**

In [32]:
korpus_negatif = ['tidak', 'ga', 'gak', 'ngga', 'nggak', 'selain', 'buruk', 'bukan', 'kurang', 'berbeda', 'beda', 'no', 'except', 'not']

In [33]:
korpus_category = ['jas', 'coat',
                   'kaos', 'shirt', 'tshirt',
                   'blouse', 'blus',
                   'glasses', 'kacamata',
                   'celana', 'pants', 'trousers',
                   'gelang', 'bracelet',
                   'dompet', 'purse',
                   'baju', 'dress',
                   'blazer',
                   'perhiasan', 'jewellery',
                   'tas', 'bag',
                   'rok', 'skirt',
                   'kemeja',
                   'sepatu', 'shoes',
                   'kalung', 'necklace',
                   'sweater',
                   'parfum', 'parfume',
                   'lipstik',
                   'pashmina', 'makeup',
                   'anting', 'earrings',
                   'jam', 'clock', 'watch',
                   'topi', 'hat',
                   'sandal',
                   'jaket', 'jacket',
                   'kemeja']

In [34]:
korpus_tag = ["circle", "bulat", "lingkaran",
              "heart","hati",
              "oblong",
              "oval",
              "square", "kotak", "segiempat",
              "triangle", "segitiga",
              "extraversion", "ekstrover", "enerjik", "terbuka", "sosial",
              "neurotic", "emosional", "labil", "cemas", "gugup",
              "agreeable", "ramah", "sopan", "bersahabat", "bergaul",
              "conscientious", "bertanggung jawab", "disiplin", "cermat", "teliti",
              "openness", "terbuka", "fleksibel", "kreatif", "inovatif"]

### **Define Query**

In [35]:
query = "AA"

### **Preprocessing Query & Extract Feature**

In [36]:
preprocess_queries = preprocess_query(query)
result_category = check_fuzzy_ratio(preprocess_queries.split(), korpus_category + korpus_tag)
separated_queries = separate_query(preprocess_queries, result_category, korpus_category + korpus_tag)
print(separated_queries)

['aa']


In [37]:
corrected_queries = [fix_typo_with_fuzzy_ratio(query, korpus_negatif) for query in separated_queries]
print(corrected_queries)

['aa']


In [38]:
keywords_cat = identify_keywords(corrected_queries, korpus_category)
keywords_tag = identify_keywords(corrected_queries, korpus_tag)
keywords_neg = identify_keywords(corrected_queries, korpus_negatif)
print(keywords_cat)
print(keywords_tag)
print(keywords_neg)

['']
['']
['']


In [39]:
neg_category_query = neg_merge_lists(keywords_neg, keywords_cat)
neg_tag_query = neg_merge_lists(keywords_neg, keywords_tag)
pos_category_query, pos_tag_query = pos_merge_lists(keywords_cat, keywords_tag, keywords_neg)
print("Negative Category :", neg_category_query)
print("Negative Tag :", neg_tag_query)
print("Positive Category :", pos_category_query)
print("Positive Tag :", pos_tag_query)

Negative Category : []
Negative Tag : []
Positive Category : []
Positive Tag : []


### **Delete Row that Contain Negative Query**

In [40]:
# Membuat fungsi untuk memeriksa apakah tag mengandung kata dalam neg_category_query
def has_negative_tags(tag, neg_query):
    for query in neg_query:
        if query in tag:
            return True
    return False

In [41]:
# Menghapus baris pada DataFrame yang memiliki category yang mengandung kata dalam neg_category_query
df = df[~df['category'].apply(lambda x: has_negative_tags(x, neg_category_query))]

In [42]:
# Menghapus baris pada DataFrame yang memiliki tag yang mengandung kata dalam neg_tag_query
df = df[~df['tags'].apply(lambda x: has_negative_tags(x, neg_tag_query))]

In [43]:
df.head()

,id,name,description,category,brand,tags,merchants
0,1,kaito natsuki signature,spesifikasi lebar frame 140 mm lebar lensa 53 ...,glasses,kcmtku,square kotak,
1,2,myki nial eyewear,spesifikasi lebar frame 146 mm lebar lensa 54 ...,glasses,kcmtku,square kotak,
2,3,kaito cleo signature,spesifikasi lebar frame 135 mm lebar lensa 53 ...,glasses,kcmtku,aviator,
3,4,kaito moana signature,spesifikasi lebar frame 138 mm lebar lensa 54 ...,glasses,kcmtku,aviator,
4,5,acetate cat eye,nomor model mg01 bahan lensa pc bahan bingkai ...,glasses,prada,cat eye,


### **Only Recommend Product that Contain Positive Query**

In [44]:
if len(pos_category_query) != 0:
    df = df[df['category'].apply(lambda x: any(tag in x for tag in pos_category_query))]

In [45]:
if len(pos_tag_query) != 0:
    df = df[df['tags'].apply(lambda x: any(tag in x for tag in pos_tag_query))]

In [46]:
df.head()

,id,name,description,category,brand,tags,merchants
0,1,kaito natsuki signature,spesifikasi lebar frame 140 mm lebar lensa 53 ...,glasses,kcmtku,square kotak,
1,2,myki nial eyewear,spesifikasi lebar frame 146 mm lebar lensa 54 ...,glasses,kcmtku,square kotak,
2,3,kaito cleo signature,spesifikasi lebar frame 135 mm lebar lensa 53 ...,glasses,kcmtku,aviator,
3,4,kaito moana signature,spesifikasi lebar frame 138 mm lebar lensa 54 ...,glasses,kcmtku,aviator,
4,5,acetate cat eye,nomor model mg01 bahan lensa pc bahan bingkai ...,glasses,prada,cat eye,


### **Build Search Engine**

In [47]:
# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

In [48]:
def search_and_get_results(query, vectorizer, top_n):
    results = []
    for idx, product in df.iterrows():
        # Menghitung similarity score untuk setiap kolom
        similarity_scores = []

        for column in ['name', 'description', 'brand', 'merchants']:
            query_ngrams = set(ngrams(query.lower(), 3))
            product_ngrams = set(ngrams(product[column].lower(), 3))

            # Check if the sets are empty before calculating similarity
            if len(query_ngrams) == 0 or len(product_ngrams) == 0:
                ngram_similarity = 0
            else:
                ngram_similarity = len(query_ngrams.intersection(product_ngrams)) / len(query_ngrams.union(product_ngrams))

            levenshtein_distance = edit_distance(query.lower(), product[column].lower())
            levenshtein_similarity = 1 - (levenshtein_distance / max(len(query), len(product[column])))

            similarity_score = (ngram_similarity + levenshtein_similarity) / 2

            # Meningkatkan bobot similarity score pada kolom "category" dan "tags"
            if column == 'name':
                similarity_score *= 2
            elif column == 'description':
                similarity_score *= 1
            elif column == 'brand':
                similarity_score *= 3
            else:
                similarity_score *= 2

            similarity_scores.append(similarity_score)

        # Menjumlahkan similarity scores dari setiap kolom
        total_similarity_score = sum(similarity_scores) / len(similarity_scores)
        results.append((product['id'], total_similarity_score))

    # Mengurutkan hasil berdasarkan similarity score
    results = sorted(results, key=lambda x: x[1], reverse=True)[:top_n]

    # Membuat DataFrame hasil pencarian
    search_results = pd.DataFrame(results, columns=['id', 'similarity'])
    return search_results


In [49]:
# Menampilkan produk berdasarkan query
search_results = search_and_get_results(query, tfidf_vectorizer, top_n=df.shape[0])
merged_df = pd.merge(df, search_results, on='id', how='inner')

# Menampilkan data frame dengan nilai similarity lebih dari 0.1
merged_df = merged_df[merged_df['similarity'] > 0.1]

# Mengeprint tidak ada produk apabila merfed_df tidak mempunyai baris
if merged_df.shape[0] == 0:
    print("Tidak ada produk yang sesuai dengan pencarian\n")

# Mengurutkan dataframe berdasarkan kolom similarity yang terbesar
merged_df = merged_df.sort_values(by=['similarity'], ascending=False)
merged_df.head(25)

,id,name,description,category,brand,tags,merchants,similarity
4,5,acetate cat eye,nomor model mg01 bahan lensa pc bahan bingkai ...,glasses,prada,cat eye,,0.185349
7,8,cazal legends men s,brand cazal legends model 663 3 style retro pi...,glasses,cazal,oversized,,0.177736
6,7,ray ban rb 4307,material frame injected form rectangular sport...,glasses,rayban,oversized,,0.160452
